In [1]:
cd lag-llama

/home/user/energygpt/lagllama/lag-llama


/home/user/miniconda3/envs/lagllama/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import glob
from collections import defaultdict
from datetime import datetime
from itertools import islice

from matplotlib import pyplot as plt
import matplotlib.dates as mdates

import torch
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset

from gluonts.dataset.pandas import PandasDataset
import pandas as pd

from lag_llama.gluon.estimator import LagLlamaEstimator

import warnings  
warnings.filterwarnings('ignore') 

In [3]:
def get_lag_llama_predictions(dataset, prediction_length, device, context_length=32, use_rope_scaling=False, num_samples=100):
    ckpt = torch.load("./checkpoints/lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we use a GPU.
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    rope_scaling_arguments = {
        "type": "linear",
        "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
    }

    estimator = LagLlamaEstimator(
        ckpt_path="./checkpoints/lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length, # Lag-Llama was trained with a context length of 32, but can work with any context length

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],
        rope_scaling=rope_scaling_arguments if use_rope_scaling else None,

        batch_size=32,
        num_parallel_samples=100,
        device=device,
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    return forecasts, tss

In [5]:
# Data pipelining
def get_batched_data_fn(sub_df,
    batch_size: int = 128, 
    context_len: int = 168, 
    horizon_len: int = 24):
    
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(sub_df) - (context_len + horizon_len), horizon_len):
      num_examples += 1
      examples["inputs"].append(sub_df["y"][start:(context_end := start + context_len)].tolist())
      examples["outputs"].append(sub_df["y"][context_end:(context_end + horizon_len)].tolist())
      examples['inputs_ts'].append(sub_df.index[start:(context_end := start + context_len)])
      examples["outputs_ts"].append(sub_df.index[context_end:(context_end + horizon_len)])

    return examples
  

In [6]:
def forecast_building(df):


    for col in df.columns:
        if df[col].dtype != 'object' and pd.api.types.is_string_dtype(df[col]) == False:
            df[col] = df[col].astype('float32')
    
    # Create the Pandas
    dataset = PandasDataset.from_long_dataframe(df.head(30*192), target="target", item_id="item_id")

    
    backtest_dataset = dataset
    prediction_length = 24  # Define your prediction length. We use 24 here since the data is of hourly frequency
    num_samples = 10 # number of samples sampled from the probability distribution for each timestep
    device = torch.device("cuda:0")  
    
    ckpt = torch.load("./checkpoints/lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we use a GPU.

    forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples=num_samples)

    evaluator = Evaluator()
    agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))     
    
    res_all = []
    for ts, fc in zip(tss, forecasts):
        res = ts[ts.index.isin(fc.index)]
        res.columns = ['y_true']
        res.insert(1, 'y_pred', fc.median)
        res_all.append(res)

    res_all_df = pd.concat(res_all).sort_index()
    return res_all_df, agg_metrics, ts_metrics 

    
def process_building(df): 
    building_name = df.columns[0]
    df.columns = ['y']
    input_data = get_batched_data_fn(df, batch_size=500)
    
    windows_all = []
    counter = 1
    for inputs_ts, inputs, outputs_ts, outputs in zip(input_data['inputs_ts'], 
                                                      input_data['inputs'], 
                                                      input_data['outputs_ts'], 
                                                      input_data['outputs']):
        
        input_df = pd.DataFrame({'timestamp': inputs_ts, 
                                 'target': inputs})
        
        output_df = pd.DataFrame({'timestamp': outputs_ts, 
                                 'target': outputs})
        combined = pd.concat([input_df, output_df], axis=0)
        combined['item_id'] = str(building_name) + '_' + str(counter)
        combined['item_id_no'] = counter
        counter += 1
        windows_all.append(combined)
        
    windows_all_df = pd.concat(windows_all)
    windows_all_df.timestamp = pd.to_datetime(windows_all_df.timestamp)
    windows_all_df.set_index('timestamp', inplace=True)

    res, agg_metrics, ts_metrics = forecast_building(windows_all_df)
    return res, agg_metrics, ts_metrics


# Benchmark
batch_size = 32
context_len = 168
horizon_len = 24

def process_file(filename):
    df = pd.read_csv(filename)
    df = df.set_index(['timestamp'])

    if df.shape[1] < 2:
        return None
        
    print(datetime.now(), df.shape, flush=True)

    res_all = []
    agg_metrics_all = []
    ts_metrics_all = []
    
    i = 0
    for building_name in df.columns:
        print(datetime.now(), i, '/', len(df.columns), building_name, flush=True)
        df1 = df[[building_name]]

        res, agg_metrics, ts_metrics = process_building(df1)
        res['building'] = building_name
        res['filename'] = filename
        res_all.append(res)
        
        ts_metrics.insert(0, 'building', building_name)
        ts_metrics.insert(0, 'filename', filename)
        ts_metrics = ts_metrics.sort_values(['forecast_start'])
        ts_metrics_all.append(ts_metrics)
        
        agg_metrics_df = pd.DataFrame([agg_metrics])
        agg_metrics_df.insert(0, 'building', building_name)
        agg_metrics_df.insert(0, 'filename', filename)
        agg_metrics_all.append(agg_metrics_df)

        i += 1
        if i % 10 == 0:
            print(datetime.now(), 'Saving...')
            res_all_df = pd.concat(res_all).round(6)
            res_all_df = res_all_df.reset_index()
            res_all_df = res_all_df.rename(columns={res_all_df.columns[0]: "timestamp" })
            res_all_df.to_csv(f'../forecasts/{dataset}/{os.path.basename(filename)}', index=False)            

            ts_metrics_all_df = pd.concat(ts_metrics_all).round(6)
            ts_metrics_all_df.to_csv(f'../results/{dataset}/ts_metrics_{os.path.basename(filename)}', index=False)            

            agg_metrics_all_df = pd.concat(agg_metrics_all).round(6)            
            agg_metrics_all_df.to_csv(f'../results/{dataset}/agg_metrics_{os.path.basename(filename)}', index=False)            
    
    
    res_all_df = pd.concat(res_all).round(6)
    res_all_df = res_all_df.reset_index()
    res_all_df = res_all_df.rename(columns={res_all_df.columns[0]: "timestamp" })
    res_all_df.to_csv(f'../forecasts/{dataset}/{os.path.basename(filename)}', index=False)            

    ts_metrics_all_df = pd.concat(ts_metrics_all).round(6)    
    ts_metrics_all_df.to_csv(f'../results/{dataset}/ts_metrics_{os.path.basename(filename)}', index=False)            

    agg_metrics_all_df = pd.concat(agg_metrics_all).round(6)   
    agg_metrics_all_df.to_csv(f'../results/{dataset}/agg_metrics_{os.path.basename(filename)}', index=False)                

    return res_all_df, ts_metrics_all_df, agg_metrics_all_df

In [8]:
files_list = glob.glob('/home/user/BuildingsBenchNREL/BuildingsBenchData/BuildingsBench/Electricity/*.csv')

dataset = 'Electricity'
os.makedirs(f'../forecasts/{dataset}/', exist_ok = True)
os.makedirs(f'../results/{dataset}/', exist_ok = True)

for filename in files_list:
    print(datetime.now(), filename)
    results = process_file(filename)

    print('')

2024-07-21 09:14:58.211462 /home/user/BuildingsBenchNREL/BuildingsBenchData/BuildingsBench/Electricity/LD2011_2014_clean=2011.csv
2024-07-21 09:14:58.380918 (8759, 156)
2024-07-21 09:14:58.381710 0 / 156 MT_124


Running evaluation: 30it [00:00, 666.57it/s]


2024-07-21 09:15:05.893995 1 / 156 MT_156


Running evaluation: 30it [00:00, 476.42it/s]


2024-07-21 09:15:12.316685 2 / 156 MT_158


...

Running evaluation: 30it [00:00, 685.00it/s]


2024-07-21 11:15:17.786418 313 / 315 MT_368


Running evaluation: 30it [00:00, 582.47it/s]


2024-07-21 11:15:23.973954 314 / 315 MT_369


Running evaluation: 30it [00:00, 688.55it/s]


## Metrics

In [9]:
#!pip install permetrics

In [10]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import root_mean_squared_log_error
from permetrics.regression import RegressionMetric

dataset = 'Electricity'
files_list = glob.glob(f'../forecasts/{dataset}/*.csv')

metrics_all_files = []

for filename in files_list:
    res = pd.read_csv(filename)
    metrics_all = []
    for (g, data) in res.groupby(['building']):
        data = data.dropna()
        data = data[data.y_pred >= 0]
        #data = [data.y_pred >= 0]
        print(g)  
        if data.empty:
            continue 
            
        rmse= root_mean_squared_error(data.y_true, data.y_pred)
        mae= mean_absolute_error(data.y_true, data.y_pred)
        mape = mean_absolute_percentage_error(data.y_true, data.y_pred)
        mse= mean_squared_error(data.y_true, data.y_pred)
        msle= mean_squared_log_error(data.y_true, data.y_pred)
        rmsle= root_mean_squared_log_error(data.y_true, data.y_pred)

        evaluator = RegressionMetric(data.y_true.to_list(), data.y_pred.to_list())
        nrmse = evaluator.normalized_root_mean_square_error()
    
        metrics = pd.DataFrame({'building_name': [g[0]], 
                           'mae': [mae],
                            'mape': [mape],
                           'mse': [mse], 'rmse': [rmse], 'msle': [msle], 'rmsle': [rmsle], 'nrmse' : [nrmse]})
        metrics_all.append(metrics)
    
    metrics_all_df = pd.concat(metrics_all)
    metrics_all_df.to_csv(f'../results/{dataset}/{os.path.basename(filename)}')

    metrics_all_df['filename'] = os.path.basename(filename)
    metrics_all_files.append(metrics_all_df)

metrics_all_files_df = pd.concat(metrics_all_files)

('MT_124',)
('MT_156',)
('MT_158',)
('MT_159',)
('MT_161',)
('MT_162',)
('MT_163',)
('MT_166',)
('MT_168',)
('MT_169',)
('MT_171',)
('MT_172',)
('MT_174',)
('MT_175',)
('MT_176',)
('MT_180',)
('MT_182',)
('MT_183',)
('MT_187',)
('MT_188',)
('MT_189',)
('MT_190',)
('MT_191',)
('MT_192',)
('MT_193',)
('MT_194',)
('MT_195',)
('MT_196',)
('MT_197',)
('MT_198',)
('MT_199',)
('MT_200',)
('MT_201',)
('MT_202',)
('MT_203',)
('MT_204',)
('MT_205',)
('MT_206',)
('MT_207',)
('MT_208',)
('MT_209',)
('MT_210',)
('MT_211',)
('MT_212',)
('MT_213',)
('MT_214',)
('MT_215',)
('MT_216',)
('MT_217',)
('MT_218',)
('MT_219',)
('MT_220',)
('MT_221',)
('MT_222',)
('MT_223',)
('MT_225',)
('MT_226',)
('MT_227',)
('MT_228',)
('MT_229',)
('MT_230',)
('MT_231',)
('MT_232',)
('MT_233',)
('MT_234',)
('MT_235',)
('MT_236',)
('MT_237',)
('MT_238',)
('MT_239',)
('MT_240',)
('MT_241',)
('MT_242',)
('MT_243',)
('MT_244',)
('MT_245',)
('MT_246',)
('MT_247',)
('MT_248',)
('MT_249',)
('MT_250',)
('MT_251',)
('MT_252',)
('MT

In [11]:
metrics_all_files_df.to_csv(f'../results/{dataset}/results_combined.csv')
metrics_all_files_df

,building_name,mae,mape,mse,rmse,msle,rmsle,nrmse,filename
0,MT_124,65.007839,0.350546,7155.653642,84.591097,0.157238,0.396532,0.940633,LD2011_2014_clean=2011.csv
0,MT_156,0.471020,0.007056,1.511434,1.229404,0.000331,0.018202,1.005238,LD2011_2014_clean=2011.csv
0,MT_158,51.569080,1.101942,4558.572209,67.517199,0.987766,0.993864,1.192899,LD2011_2014_clean=2011.csv
0,MT_159,35.381122,0.652510,2240.640775,47.335407,0.750592,0.866367,1.215004,LD2011_2014_clean=2011.csv
0,MT_161,140.078922,0.069656,33237.137934,182.310554,0.007960,0.089220,1.920433,LD2011_2014_clean=2011.csv
...,...,...,...,...,...,...,...,...,...
0,MT_365,29.906176,0.710771,1594.419572,39.930184,0.478427,0.691684,0.946504,LD2011_2014_clean=2012.csv
0,MT_366,8.224463,0.482987,110.338931,10.504234,0.528370,0.726891,1.047802,LD2011_2014_clean=2012.csv
0,MT_367,70.691227,0.155165,8796.776540,93.791133,0.036897,0.192086,2.051892,LD2011_2014_clean=2012.csv
0,MT_368,52.526958,0.461500,5261.063662,72.533190,0.847326,0.920503,1.080018,LD2011_2014_clean=2012.csv
